In [1]:
#Import modules
import pandas as pd
import csv
import numpy as np
import cartopy
import matplotlib.pyplot as plt
from metpy.plots import USCOUNTIES

In [2]:
#Define  filepath variables to be used later for reading/writing files
user = ''
directory = '\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\Balikian\\ISWS_HydroGeo\\MetroEast_HydroGeo\\WellData\\'
wells_filename = 'Statewide_ClassifiedWells.txt'
wellLocs_filename = 'Statewide_WellLocs.txt'
encodeType = 'latin1'

In [4]:
#Read in well description data to pandas dataframe
wells = pd.read_csv(directory+wells_filename, sep=',', header='infer', encoding=encodeType) #Not working for whatever reason, so use below code
wells

,API_Number,TOP,BOTTOM,FORMATION_SHORT,LITHOLOGY,COLOR,CONSISTENCY,MOD1,MOD2,INTERPRETED,COMPLETED
0,121114109700,10.0,36.0,gravel,GRAVEL,NaN,NaN,NaN,NaN,GRAVEL,1.0
1,121114109700,36.0,98.0,clay,CLAY,NaN,NaN,NaN,NaN,CLAY,1.0
2,121114109700,98.0,103.0,sand w/gravel,SAND WITH GRAVEL,NaN,NaN,NaN,NaN,GRAVEL,-1.0
3,121114109700,103.0,237.0,clay,CLAY,NaN,NaN,NaN,NaN,CLAY,1.0
4,121114109700,237.0,250.0,limestone,BEDROCK,NaN,NaN,LIMESTONE,SHALE,BEDROCK,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1824950,480590602600,19.0,279.0,gravel & dirty sand,SAND AND GRAVEL,NaN,NaN,DIRTY,NaN,GRAVEL,-1.0
1824951,480590602600,20.0,20.0,topsoil & clay,CLAY,NaN,NaN,NaN,NaN,CLAY,-1.0
1824952,480590602600,20.0,260.0,red clay & gravel,CLAY AND GRAVEL,RED,NaN,NaN,NaN,CLAY AND GRAVEL MIX,-1.0
1824953,480590602600,60.0,240.0,fine sand,SAND,NaN,NaN,NaN,NaN,SAND,1.0


In [ ]:
#No longer needed. Used for work around when Pandas read-in wasn't working due to encoding issue
#wellData = []
#with open(directory+wells_filename) as wellFile:
#    wells = csv.reader(wellFile, delimiter=',')
#    for row in wells:
#        wellData.append(row)
#cols = wellData[0]
#del wellData[0]
#wells = pd.DataFrame(wellData)
#wells.columns = cols
#wells

In [5]:
#Read in well location data to pandas dataframe
wellLocs = pd.read_csv(directory+wellLocs_filename, sep=',')
wellLocs

,ID,API_Number,LATITUDE,LONGITUDE,SURF_ELEV_ft
0,0,120333071500,38.923294,-87.737657,603.67
1,1,120333072100,38.896763,-87.746863,513.81
2,2,120390005600,40.276271,-88.814961,770.33
3,3,120390006800,40.279958,-88.810223,772.36
4,4,120390040300,40.276241,-88.860433,799.78
...,...,...,...,...,...
281510,281510,121152386100,39.910254,-88.894730,679.72
281511,281511,121152386200,39.918670,-88.835602,618.79
281512,281512,121152386300,39.913466,-88.813955,631.60
281513,281513,121152386400,39.965009,-88.774931,676.63


In [ ]:
#Recast API Number column as floats for size and to ensure they match between dataframes
wells['API_Number'] = wells['API_Number'].astype(float)
wellLocs['API_Number'] = wellLocs['API_Number'].astype(float)

In [ ]:
#Merge the two dataframe to get descriptions and locations into one dataframe
wellswithLocs = wells.merge(wellLocs, how='inner', on='API_Number')

In [ ]:
#Replace empty cells in top/bottom columns with nans to aid later processing
wellswithLocs['TOP'] = wellswithLocs['TOP'].replace('', np.nan)
wellswithLocs['BOTTOM'] = wellswithLocs['BOTTOM'].replace('', np.nan)

In [ ]:
#Convert necessary columns to float
wellswithLocs['TOP'] = wellswithLocs['TOP'].astype(float)
wellswithLocs['BOTTOM'] = wellswithLocs['BOTTOM'].astype(float)

wellswithLocs['LATITUDE'] = wellswithLocs['LATITUDE'].astype(float)
wellswithLocs['LONGITUDE'] = wellswithLocs['LONGITUDE'].astype(float)

In [ ]:
#Get well interval elevations in feet from surface elevation depth
wellswithLocs['TOP_ELEV_ft'] = wellswithLocs['SURF_ELEV_ft'] - wellswithLocs['TOP']
wellswithLocs['BOT_ELEV_ft'] = wellswithLocs['SURF_ELEV_ft'] - wellswithLocs['BOTTOM']

In [ ]:
#Quality Checks
#View new dataframe
wellswithLocs

In [ ]:
#Chart data 
proj = cartopy.crs.PlateCarree()

fig, ax = plt.subplots(1, figsize=(15,15), subplot_kw=dict(projection=proj))
ax.scatter(wellswithLocs['LONGITUDE'], wellswithLocs['LATITUDE'])
ax.add_feature(cartopy.feature.STATES)
ax.add_feature(USCOUNTIES)
ax.set_extent([-90.5,-89.5,38,39.25])

In [ ]:
#Export data to new data frame
wellswithLocs.to_csv(directory+'Statewide_Wells_XYZ.txt')